In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.cluster import KMeans,MiniBatchKMeans
from sklearn.preprocessing import RobustScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

properati = pd.read_csv('datos/properati_final.csv',error_bad_lines=False)

<IPython.core.display.Javascript object>

In [12]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['place_name']
le_barrio.fit(barrios)
properati['place_name'] = le_barrio.transform(barrios)

le_barrio = preprocessing.LabelEncoder()
barrios=properati['state_name']
le_barrio.fit(barrios)
properati['state_name'] = le_barrio.transform(barrios)

le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)

### Nota: Kmeans llama inertia a la funcion costo/distorsion

In [13]:
X= zip(properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['property_type'],properati['state_name'],properati['place_name'])

In [24]:
%%notify

km = KMeans(n_jobs=-1)

n_clusters  = np.arange(30,300,1)

X_train, X_test= train_test_split(X, test_size=0.2)

param_grid = {"n_clusters": n_clusters}

search = RandomizedSearchCV(km, param_distributions=param_grid ,cv=5,n_iter=100) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

Process PoolWorker-2498:
Process PoolWorker-2500:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    self._target(*self._args, **self._kwargs)
    task = get()
  File "/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/pool.py", line 360, in get
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 102, in worker


KeyboardInterrupt: 

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [14]:
%%notify

mini = MiniBatchKMeans()

n_clusters  = np.arange(30,300,1)

X_train, X_test= train_test_split(X, test_size=0.2)

param_grid = {"n_clusters": n_clusters}

search = RandomizedSearchCV(mini, param_distributions=param_grid ,cv=5,n_iter=20) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 111.86 segundos para 20 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: -98555360034.037 (std: 39473461340.494)
Promedio validation score: -22170377164.865 (std: 4472072665.595)
Promedio fit time: 0.879s
Hiper-parametros: {'n_clusters': 258}

Puesto: 2
Promedio training score: -95454434129.177 (std: 27002164837.522)
Promedio validation score: -22565212903.161 (std: 14557446613.323)
Promedio fit time: 0.743s
Hiper-parametros: {'n_clusters': 198}

Puesto: 3
Promedio training score: -95098511398.002 (std: 35838315136.093)
Promedio validation score: -24926573581.343 (std: 10972382317.449)
Promedio fit time: 0.595s
Hiper-parametros: {'n_clusters': 140}

Puesto: 4
Promedio training score: -96342547072.069 (std: 25844817927.991)
Promedio validation score: -25155287391.977 (std: 16350747483.216)
Promedio fit time: 1.410s
Hiper-parametros: {'n_clusters': 291}

Puesto: 5
Promedio training score: -102014039777.659 (std: 21450349300.066)
Promedio validation sco

<IPython.core.display.Javascript object>

In [16]:
search.best_estimator_.inertia_

133255040128.52414

In [17]:
search.best_estimator_.predict(X)

array([108, 116,  65, ..., 164, 226, 245], dtype=int32)